In [ ]:
# Next step!
# See if you can stop deallocation, or look at pinning memory, or mapping memory
# defer cleanup - test with smaller arrays to see how this works

In [1]:
import numpy as np
import pandas as pd
import fabio
#import cupy as cp
from datetime import datetime
from numba import jit, cuda, prange
import time
import math
# import numba

In [2]:
#initialze detector info
# //These are the details of the detector in the detInfo wave
# 	//detInfo[0] = xdim
# 	//detInfo[1] = ydim
# 	//detInfo[2] = xpixsize	Given in mm
# 	//detInfo[3] = ypixsize	Given in mm
# 	//detInfo[4] = xcen (pixels)
# 	//detInfo[5] = ycen (pixels)
# 	//detInfo[6] = tiltang	Given in degrees
# 	//detInfo[7] = tiltrot		Given in degrees
# 	//detInfo[8] = sample to detector distance	Given in mm
# 	//detInfo[9] = lamda	Given in Angstroms
# detInfo = np.array([1475,1679,0.17200001,0.17200001,738.00598,829.4032,0,0,450,0.23946001])
detInfo = np.array([2048,2048,0.085,0.085,1062.08,1042.1,-0.655953,133.021,714.029,0.16118])
imFilePrefix = ""
imNumbers = np.arange(1,1000)
boxsize = 501
qmax = 5
# boxname = ""
omegaPos = np.arange(9.8,-50.2,-0.2)
phiPos = np.zeros_like(omegaPos)
psiPos = np.zeros_like(omegaPos)
mask = []
rejects = []
cust_mask = []
mask_list = []
rotmatrix = np.array([0.61190927,0.098435618,-0.78477865,-0.78786123,-0.011431515,-0.61574668,0.069582619,-0.99507779,-0.070558563]).reshape((3,3))
normInt = np.arange(1,1000)
startIm = 0
# box = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)

In [131]:
def DiffuseReconstruction(fileNum):
    # t0 = time.perf_counter()
    tiltang = detInfo[6]*np.pi/180; tiltrot = detInfo[7]*np.pi/180; dist = detInfo[8]
    
#     row,col = np.indices((int(detInfo[0]),int(detInfo[1])))
    row = np.arange(0,int(detInfo[0])).repeat(int(detInfo[1])).reshape(int(detInfo[0]),int(detInfo[1]))
    col = np.arange(0,int(detInfo[1])).repeat(int(detInfo[0])).reshape(int(detInfo[1]),int(detInfo[0]))
    col = np.transpose(col)
#     TT = (np.arctan(np.sqrt((np.cos(tiltang)**2*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3]))**2 + (-np.sin(tiltrot)*((row-detInfo[4])*detInfo[2]) + np.cos(tiltrot)*((col-detInfo[5])*detInfo[3]))**2)/((dist+np.sin(tiltang)*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3])))**2))))/2
    TT = (np.sqrt(np.arctan((np.cos(tiltang)**2*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3]))**2 + (-np.sin(tiltrot)*((row-detInfo[4])*detInfo[2]) + np.cos(tiltrot)*((col-detInfo[5])*detInfo[3]))**2)/((dist+np.sin(tiltang)*(np.cos(tiltrot)*((row-detInfo[4])*detInfo[2])+np.sin(tiltrot)*((col-detInfo[5])*detInfo[3])))**2))))/2
    AZ = np.arctan2(((col-detInfo[5])*detInfo[3]), ((row-detInfo[4])*detInfo[2]))+np.pi 
# #     TT = np.transpose(TT)
# #     AZ = np.transpose(AZ)
    
    # TT1D = np.reshape(TT,int(detInfo[0])*int(detInfo[1]))
    # AZ1D = np.reshape(AZ,int(detInfo[0])*int(detInfo[1]))

    bigbox = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)
    numpixels = np.zeros((boxsize,boxsize,boxsize),dtype=np.uint32)
    # Use ravel() for shallow copy or flatten() for deep copy

    # BBposX = np.zeros((int(detInfo[0]),int(detInfo[1])),dtype=np.int16)
    # BBposY = np.zeros((int(detInfo[0]),int(detInfo[1])),dtype=np.int16)
    # BBposZ = np.zeros((int(detInfo[0]),int(detInfo[1])),dtype=np.int16)

    qrangex = np.linspace(-(qmax),qmax,boxsize)
    qrangey = np.linspace(-(qmax),qmax,boxsize)
    qrangez = np.linspace(-(qmax),qmax,boxsize)

    # Make/copy arrays to gpu device
    d_TT = cuda.to_device(TT)
    d_AZ = cuda.to_device(AZ)
    d_qx = cuda.device_array_like(d_TT)
    d_qy = cuda.device_array_like(d_TT)
    d_qz = cuda.device_array_like(d_TT)
    d_rotmatrix = cuda.to_device(rotmatrix)
    d_rotated_qx = cuda.device_array_like(d_TT)
    d_rotated_qy = cuda.device_array_like(d_TT)
    d_rotated_qz = cuda.device_array_like(d_TT)
    d_BBposX = cuda.device_array_like(d_TT)
    d_BBposY = cuda.device_array_like(d_TT)
    d_BBposZ = cuda.device_array_like(d_TT)
    #set the output file from the CUDA kernel
    d_bigboxlooper = cuda.device_array_like(bigbox)
    d_numpixelslooper = cuda.device_array_like(numpixels)
    #pin the memory for the image to be loaded into the CUDA kernel
    pinned_CI = cuda.pinned_array(TT.shape)

    qrangexstart = qrangex[0]
    qrangexdelta = qrangex[1]
    qrangeystart = qrangey[0]
    qrangeydelta = qrangey[1]
    qrangezstart = qrangez[0]
    qrangezdelta = qrangez[1]
    wavelength = detInfo[9]
    detector_xdim = detInfo[0]
    detector_ydim = detInfo[1]

    # CUDA variables
    # The number of threads per block should be a round multiple of the warp size, which is 32 on all current hardware.
    blocks = (128,128)
    threads_per_block = (24,24)
    # blocks = (64,64)
    # threads_per_block = (16,16)

    # remember to change this so it's more robust
    bigbox_dims = bigbox.shape[0]

    # t1p5 = time.perf_counter() - t0
    # print(t1p5)

    for i in range(fileNum):
        # t1 = time.perf_counter()
        # if i % 10 == 0:
        print(i)
        #use FABIO to load the image
        CI_fabio = fabio.open("C:\\Users\\Patrick\\Desktop\\s1_363K_2p0_diffuse\\DisCorr\\s1_363K_2p0_diffuse_"+("%04d" % (i))+".edf")
        #convert to numpy array
        # pinned_CI = np.asarray(CI_fabio.data)
        pinned_CI = CI_fabio.data
#         CI = np.transpose(CI)
        # CI = np.reshape(CI,int(detInfo[0])*int(detInfo[1]))
        # pinned_CI = np.reshape(CI,int(detInfo[0])*int(detInfo[1]))
        # pinned_CI = CI
        # print(pinned_CI[2048*1050])
        
        omega = omegaPos[i]*np.pi/180; phi = phiPos[i]*np.pi/180; psi = psiPos[i]*np.pi/180
        # row_dim = CI.shape[0]

        # qx = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (np.sin(TT1D)*(np.sin(phi)*np.sin(psi)*np.sin(omega) + np.cos(phi)*np.cos(omega)) + np.cos(TT1D)*np.cos(AZ1D)*np.sin(phi)*np.cos(psi) - np.cos(TT1D)*np.sin(AZ1D)*(np.sin(phi)*np.sin(psi)*np.cos(omega) - np.cos(phi)*np.sin(omega)))
        # qy = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (-np.sin(TT1D)*(np.cos(phi)*np.sin(psi)*np.sin(omega) - np.sin(phi)*np.cos(omega)) - np.cos(TT1D)*np.cos(AZ1D)*np.cos(phi)*np.cos(psi) + np.cos(TT1D)*np.sin(AZ1D)*(np.cos(phi)*np.sin(psi)*np.cos(omega) + np.sin(phi)*np.sin(omega)))
        # qz = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (np.sin(TT1D)*np.cos(psi)*np.sin(omega) - np.cos(TT1D)*np.sin(AZ1D)*np.cos(psi)*np.cos(omega) - np.cos(TT1D)*np.cos(AZ1D)*np.sin(psi))
    #         qz = (4*np.pi*np.sin(TT1D)/detInfo[9]) * (np.sin(TT1D)*np.cos(psi)*np.sin(omega) - np.cos(TT1D)*np.sin(AZ1D)*np.cos(psi)*np.cos(omega) - np.cos(TT1D)*np.cos(AZ1D)*np.sin(psi))

        # apply matrix rotation
        # qxyz = np.concatenate((qx,qy,qz)).reshape(3,qx.size)
        # qxyz = rotmatrix @ qxyz
        
        # BBposX = np.around((qxyz[0] - (qrangex[0])) / (qrangex[1]-qrangex[0]))
        # BBposY = np.around((qxyz[1] - (qrangey[0])) / (qrangey[1]-qrangey[0]))
        # BBposZ = np.around((qxyz[2] - (qrangez[0])) / (qrangez[1]-qrangez[0]))
        
        # t2 = time.perf_counter() - t1
        # print(t2)

        # move the variables required to the device
        # d_omega = cuda.to_device(omega)
        # d_phi = cuda.to_device(phi)
        # d_psi = cuda.to_device(psi)
#         d_i = cuda.to_device(i)
        d_CI = cuda.to_device(pinned_CI)
        # d_CI = cuda.to_device(CI)

#         d_qmax = cuda.to_device(qmax)
#         d_qrangex = cuda.to_device(qrangex)
#         d_qrangey = cuda.to_device(qrangey)
#         d_qrangez = cuda.to_device(qrangez)
        # d_BBposX = cuda.to_device(BBposX)
        # d_BBposY = cuda.to_device(BBposY)
        # d_BBposZ = cuda.to_device(BBposZ)
        # d_bigboxlooper = cuda.device_array_like(bigbox)
        # d_numpixelslooper = cuda.device_array_like(numpixels)
        
        # t3 = time.perf_counter() - t2
        # print('copying to device takes', t3)

        # t4 = time.perf_counter()

        looper[blocks,threads_per_block](detector_xdim, detector_ydim, bigbox_dims, omega, phi, psi, wavelength, qrangexstart, qrangexdelta, qrangeystart, qrangeydelta, qrangezstart, qrangezdelta, d_TT, d_AZ, d_CI, d_qx, d_qy, d_qz, d_rotmatrix, d_rotated_qx, d_rotated_qy, d_rotated_qz, d_BBposX, d_BBposY, d_BBposZ, d_bigboxlooper, d_numpixelslooper)
        # t5 = time.perf_counter() - t4
        # print('loop takes ', t5)
        # bigboxlooper = d_bigboxlooper.copy_to_host()
        # numpixelslooper = d_numpixelslooper.copy_to_host()
        
        # t6 = time.perf_counter() - t5
        # print('copying to host takes ', t6)

        # bigbox += bigboxlooper
        # numpixels += numpixelslooper

        # t7 = time.perf_counter() - t6
        # print(t7)
    
    # print(d_qx.copy_to_host())
    # print(d_qy.copy_to_host())
    # print(d_qz.copy_to_host())
    # print(d_rotated_qx.copy_to_host())
    # print(d_rotated_qy.copy_to_host())
    # print(d_rotated_qz.copy_to_host())
    bigbox = d_bigboxlooper.copy_to_host()
    numpixels = d_numpixelslooper.copy_to_host()
    # bigbox += bigboxlooper
    # numpixels += numpixelslooper
    #ignore dividing by inf or 0
    np.seterr(divide='ignore', invalid='ignore')
    bigbox /= numpixels
    
    return bigbox, numpixels

In [132]:
@cuda.jit
def looper(detector_xdim, detector_ydim, bigbox_dims, omega, phi, psi, wavelength, qrangexstart, qrangexdelta, qrangeystart, qrangeydelta, qrangezstart, qrangezdelta, TT, AZ, CI, qx, qy, qz, rotmatrix, rotated_qx, rotated_qy, rotated_qz, BBposX, BBposY, BBposZ, bigboxlooper, numpixelslooper):
    grid_row, grid_col = cuda.grid(2)
    stride_row, stride_col = cuda.gridsize(2)
    
    for data_row in range(grid_row,detector_xdim,stride_row):
        for data_col in range(grid_col,detector_ydim,stride_col):

            qx[data_row][data_col] = (4*math.pi*math.sin(TT[data_row][data_col])/wavelength) * (math.sin(TT[data_row][data_col])*(math.sin(phi)*math.sin(psi)*math.sin(omega) + math.cos(phi)*math.cos(omega)) + math.cos(TT[data_row][data_col])*math.cos(AZ[data_row][data_col])*math.sin(phi)*math.cos(psi) - math.cos(TT[data_row][data_col])*math.sin(AZ[data_row][data_col])*(math.sin(phi)*math.sin(psi)*math.cos(omega) - math.cos(phi)*math.sin(omega)))
            qy[data_row][data_col] = (4*math.pi*math.sin(TT[data_row][data_col])/wavelength) * (-math.sin(TT[data_row][data_col])*(math.cos(phi)*math.sin(psi)*math.sin(omega) - math.sin(phi)*math.cos(omega)) - math.cos(TT[data_row][data_col])*math.cos(AZ[data_row][data_col])*math.cos(phi)*math.cos(psi) + math.cos(TT[data_row][data_col])*math.sin(AZ[data_row][data_col])*(math.cos(phi)*math.sin(psi)*math.cos(omega) + math.sin(phi)*math.sin(omega)))
            qz[data_row][data_col] = (4*math.pi*math.sin(TT[data_row][data_col])/wavelength) * (math.sin(TT[data_row][data_col])*math.cos(psi)*math.sin(omega) - math.cos(TT[data_row][data_col])*math.sin(AZ[data_row][data_col])*math.cos(psi)*math.cos(omega) - math.cos(TT[data_row][data_col])*math.cos(AZ[data_row][data_col])*math.sin(psi))

            rotated_qx[data_row][data_col] = rotmatrix[0][0] * qx[data_row][data_col] + rotmatrix[0][1] * qy[data_row][data_col] + rotmatrix[0][2] * qz[data_row][data_col]
            rotated_qy[data_row][data_col] = rotmatrix[1][0] * qx[data_row][data_col] + rotmatrix[1][1] * qy[data_row][data_col] + rotmatrix[1][2] * qz[data_row][data_col]
            rotated_qz[data_row][data_col] = rotmatrix[2][0] * qx[data_row][data_col] + rotmatrix[2][1] * qy[data_row][data_col] + rotmatrix[2][2] * qz[data_row][data_col]

            BBposX[data_row][data_col] = round((rotated_qx[data_row][data_col] - (qrangexstart)) / (qrangexdelta-qrangexstart))
            BBposY[data_row][data_col] = round((rotated_qy[data_row][data_col] - (qrangeystart)) / (qrangeydelta-qrangeystart))
            BBposZ[data_row][data_col] = round((rotated_qz[data_row][data_col] - (qrangezstart)) / (qrangezdelta-qrangezstart))
        
            if(0 <= BBposX[data_row][data_col] and 0 <= BBposY[data_row][data_col] and 0 <= BBposZ[data_row][data_col] and BBposX[data_row][data_col] < bigbox_dims and BBposY[data_row][data_col] < bigbox_dims and BBposZ[data_row][data_col] < bigbox_dims ):
                cuda.atomic.add(bigboxlooper, (BBposX[data_row][data_col], BBposY[data_row][data_col], BBposZ[data_row][data_col]), CI[data_row][data_col])
                cuda.atomic.add(numpixelslooper, (BBposX[data_row][data_col], BBposY[data_row][data_col], BBposZ[data_row][data_col]), 1)

                
                    # if(np.isfinite(CI[j])):

    

In [135]:
import gc
#garbage collector
gc.collect()

startTime = datetime.now()
box,num = DiffuseReconstruction(300)
print(datetime.now() - startTime)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [136]:
from matplotlib import pyplot       # Load matplotlib
%matplotlib qt 
pyplot.imshow(box[:, :, 250]) 
pyplot.show() 


In [26]:
box[250, 250, 260]

nan

In [7]:
import winsound
winsound.Beep(300, 750)

In [27]:
%reload_ext line_profiler

%lprun -f looper

Timer unit: 1e-07 s

In [28]:
        import cProfile

        cProfile.run('DiffuseReconstruction(10)')

_datetime_dtype)
       60    0.000    0.000    0.000    0.000 driver.py:1794(_workaround_for_datetime)
       60    0.000    0.000    0.001    0.000 driver.py:1802(host_pointer)
       60    0.000    0.000    0.000    0.000 driver.py:1827(memory_size_from_info)
      120    0.000    0.000    0.001    0.000 driver.py:1844(device_pointer)
      120    0.000    0.000    0.001    0.000 driver.py:1849(device_ctypes_pointer)
      780    0.000    0.000    0.001    0.000 driver.py:1857(is_device_memory)
      120    0.000    0.000    0.000    0.000 driver.py:1868(require_device_memory)
       40    0.000    0.000    0.261    0.007 driver.py:1885(host_to_device)
       20    0.000    0.000    4.580    0.229 driver.py:1903(device_to_host)
      390    6.593    0.017    6.594    0.017 driver.py:290(safe_cuda_api_call)
      390    0.001    0.000    0.001    0.000 driver.py:318(_check_error)
      300    0.000    0.000    0.000    0.000 driver.py:364(get_active_context)
      300    0.001    0.0

In [108]:
startTime = datetime.now()

box,num = DiffuseReconstruction(300)

print(datetime.now() - startTime)

0:01:13.552489


<function numba.cuda.api.list_devices()>

In [27]:
import numba
numba.cuda.is_available()

True